In [32]:
import geopandas as gpd
import pandas as pd
import duckdb
import os

In [33]:
rep = 'files_ocsge'
liste_files = os.listdir(rep)

In [34]:
#liste_files = liste_files[62:]
liste_files = [file for file in liste_files if file[26:28] in ['30']] 

In [35]:
duckdb.execute('load spatial;')

In [ ]:
for file in liste_files:
    duckdb.sql("CREATE OR REPLACE VIEW ocs_ge AS from read_parquet('{}/{}')".format(rep,file))
    duckdb.sql("create or replace table occ_sol as select *  from ocs_ge;")
    duckdb.sql("create or replace table occ_sol as " 
           "select * EXCLUDE (geometry) , ST_Transform (geometry, 'EPSG:2154', 'EPSG:3035') as geometry "
           "from occ_sol")
    duckdb.sql("create index sindex on occ_sol USING RTREE(geometry);")
    duckdb.sql("create or replace table etendue as "  
           "select st_Xmin(geometry) as xmin, st_Xmax(geometry) as xmax, st_Ymin(geometry) as ymin, st_Ymax(geometry) as ymax "
           "from (select ST_Extent_Agg(geometry) as geometry from occ_sol);")

    duckdb.sql("""
    create or replace table rowdep as
    select * EXCLUDE (geometry) , ST_Transform (geometry, 'EPSG:2154', 'EPSG:3035') as geometry from read_parquet('ign_dep_det.parquet') 
    where code_insee=='""" + file[26:28] + """'
    """)
    
    duckdb.sql("set variable size = 1000;"
           "create or replace table carreaux_size as " 
           "select x,y, ST_GeomFromText('POLYGON(('||x||' '||y||',"
                                       "'||x + getvariable('size')||' '||y||', "  
                                       "'||x + getvariable('size')||' '||y+getvariable('size')||', "
                                       "'||x||' '||y+getvariable('size')||', "
                                       "'||x||' '||y||'))') as geometry "
           "from "
           "(select unnest(generate_series(cast(floor(xmin/getvariable('size'))*getvariable('size') as int), cast(floor(xmax/getvariable('size'))*getvariable('size') as int), getvariable('size'))) as x from etendue), "
           "(select unnest(generate_series(cast(floor(ymin/getvariable('size'))*getvariable('size') as int), cast(floor(ymax/getvariable('size'))*getvariable('size') as int), getvariable('size'))) as y from etendue);")


    
    duckdb.sql(
    """create or replace table carreaux_size as
    SELECT
        x,y,
        a.geometry AS geometry
    FROM carreaux_size a
    JOIN rowdep as b
    ON ST_Within(a.geometry, b.geometry)
    UNION
    SELECT
        x,y,
        ST_Intersection(a.geometry, b.geometry) AS geometry
    FROM carreaux_size a
    JOIN rowdep as b
    ON ST_Intersects(a.geometry, b.geometry)   
    WHERE NOT ST_Within(a.geometry, b.geometry)
    
    """)
    
    
    
    duckdb.sql("create or replace table occupation_carroye as " 
           "select x, y, CODE_CS, CODE_US, sum(st_area(st_intersection(occ_sol.geometry, carreaux_size.geometry))) as surface "
           "from occ_sol, carreaux_size " 
           "where ST_Intersects(occ_sol.geometry, carreaux_size.geometry) "
           "group by x,y,CODE_CS,CODE_US;")
    duckdb.sql("COPY occupation_carroye TO 'carreaux/carreaux_"+ file[26:28] + ".csv'")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [6]:
duckdb.sql('select * from occ_sol limit 10').df()

,id,code_cs,code_us,millesime,source,ossature,id_origine,code_or,bbox,geometry,"st_transform(geometry, 'EPSG:2154', 'EPSG:3035')"
0,OCSGE0000000010012162241,CS2.2.1,US1.1,2021,calcul,0,NC,NC,"{'xmin': 616564.12, 'ymin': 6879939.4, 'xmax':...","[5, 4, 0, 0, 0, 0, 0, 0, 65, 135, 22, 73, 134,...","[5, 4, 0, 0, 0, 0, 0, 0, 155, 168, 49, 74, 39,..."
1,OCSGE0000000010012162333,CS2.2.1,US1.1,2021,calcul,0,NC,NC,"{'xmin': 616706.0, 'ymin': 6879948.31, 'xmax':...","[5, 4, 0, 0, 0, 0, 0, 0, 32, 144, 22, 73, 152,...","[5, 4, 0, 0, 0, 0, 0, 0, 139, 168, 49, 74, 88,..."
2,OCSGE0000000010012159499,CS1.1.1.1,US1.1,2021,calcul,0,NC,NC,"{'xmin': 616667.1, 'ymin': 6879973.6, 'xmax': ...","[5, 4, 0, 0, 0, 0, 0, 0, 177, 141, 22, 73, 203...","[5, 4, 0, 0, 0, 0, 0, 0, 1, 169, 49, 74, 199, ..."
3,OCSGE0000000010012162031,CS2.1.1.1,US3,2021,calcul,0,NC,NC,"{'xmin': 617572.4, 'ymin': 6880390.4, 'xmax': ...","[5, 4, 0, 0, 0, 0, 0, 0, 70, 198, 22, 73, 12, ...","[5, 4, 0, 0, 0, 0, 0, 0, 224, 173, 49, 74, 190..."
4,OCSGE0000000010012161633,CS2.1.1.1,US3,2021,calcul,0,NC,NC,"{'xmin': 617601.4, 'ymin': 6880375.6, 'xmax': ...","[5, 4, 0, 0, 0, 0, 0, 0, 22, 200, 22, 73, 239,...","[5, 4, 0, 0, 0, 0, 0, 0, 201, 173, 49, 74, 243..."
5,OCSGE0000000010012162345,CS2.1.1.1,US3,2021,calcul,0,NC,NC,"{'xmin': 617583.78, 'ymin': 6880486.72, 'xmax'...","[5, 4, 0, 0, 0, 0, 0, 0, 252, 198, 22, 73, 205...","[5, 4, 0, 0, 0, 0, 0, 0, 48, 175, 49, 74, 13, ..."
6,OCSGE0000000010012161639,CS2.2.1,US1.1,2021,calcul,0,NC,NC,"{'xmin': 617860.0, 'ymin': 6880529.59, 'xmax':...","[5, 4, 0, 0, 0, 0, 0, 0, 64, 216, 22, 73, 35, ...","[5, 4, 0, 0, 0, 0, 0, 0, 206, 175, 49, 74, 50,..."
7,OCSGE0000000010012161110,CS1.1.1.1,US3,2021,calcul,0,NC,NC,"{'xmin': 617753.5, 'ymin': 6880586.5, 'xmax': ...","[5, 4, 0, 0, 0, 0, 0, 0, 152, 209, 22, 73, 149...","[5, 4, 0, 0, 0, 0, 0, 0, 232, 176, 49, 74, 150..."
8,OCSGE0000000010012160525,CS2.1.1.1,US3,2021,calcul,0,NC,NC,"{'xmin': 617499.52, 'ymin': 6880683.8, 'xmax':...","[5, 4, 0, 0, 0, 0, 0, 0, 184, 193, 22, 73, 87,...","[5, 4, 0, 0, 0, 0, 0, 0, 203, 178, 49, 74, 209..."
9,OCSGE0000000010012161164,CS1.1.1.1,US3,2021,calcul,0,NC,NC,"{'xmin': 617447.47, 'ymin': 6880723.64, 'xmax'...","[5, 4, 0, 0, 0, 0, 0, 0, 119, 190, 22, 73, 167...","[5, 4, 0, 0, 0, 0, 0, 0, 123, 179, 49, 74, 13,..."
